# Fun with Actions

Actions are intended for a long-term relationships and are generally high-level.

Structure of action is similar to service but provides more opportunities to communicate.

Also, during action execution server can provide feedback as well as explicitly fail

![state machine](http://wiki.ros.org/actionlib/DetailedDescription?action=AttachFile&do=get&target=server_states_detailed.png)

## Writing action server

We prepared a simple action for the Ono robot. Let's make him help with the house work

In [1]:
!cat ../../action/working.action
!cd ../../../../ && catkin_make

#goal definition
string[] list_of_orders
---
#result
string[] finished_tasks
---
#feedback
string current_task
Base path: /home/ubuntu/catkin_ws
Source space: /home/ubuntu/catkin_ws/src
Build space: /home/ubuntu/catkin_ws/build
Devel space: /home/ubuntu/catkin_ws/devel
Install space: /home/ubuntu/catkin_ws/install
####
#### Running command: "make cmake_check_build_system" in "/home/ubuntu/catkin_ws/build"
####
####
#### Running command: "make -j8 -l8" in "/home/ubuntu/catkin_ws/build"
####
[  0%] Built target actionlib_msgs_generate_messages_cpp
[  0%] Built target std_msgs_generate_messages_cpp
[  0%] Built target sensor_msgs_generate_messages_cpp
[  0%] Built target _ipython_robot_prototyping_generate_messages_check_deps_workingFeedback
[  0%] Built target _ipython_robot_prototyping_generate_messages_check_deps_Question
[  0%] Built target _ipython_robot_prototyping_generate_messages_check_deps_workingActionGoal
[  0%] Built target _ipython_robot_prototyping_generate_messages_check_d

The action takes list of orders (hopefully) manages to finish one after another
and then finishes



In [1]:
import ipython_robot_prototyping.msg

import rospy
#from __future__ import print_function

# Brings in the SimpleActionClient
import actionlib

class OnoWorker(object):
    _feedback = ipython_robot_prototyping.msg.workingFeedback()
    _result = ipython_robot_prototyping.msg.workingResult()
    
    def __init__(self,name):
        self._action_name = name
        self._action_server = actionlib.SimpleActionServer(self._action_name, 
                                                           ipython_robot_prototyping.msg.workingAction,
                                                           execute_cb=self.execute_cb,
                                                           auto_start = False)
        self._as= self._action_server
        
        self._action_server.start()
        
    def execute_cb(self, goal):
        # helper variables
        r = rospy.Rate(1)
        success = True
        rospy.loginfo("Received list of goals")
        # append the seeds for the fibonacci sequence
        self._todo = goal.list_of_orders.copy() # we start by having list full of actions
        
 
                
        
        rospy.loginfo("Starting task execution")
        
        for task in self._todo:
             
            if self._as.is_preempt_requested(): # cancel requested
                rospy.loginfo('%s: Preempted' % self._action_name)
                self._as.set_preempted()
                success = False
                return 
                #break
            else:
                self._feedback.current_task=task
                self._as.publish_feedback(self._feedback)
            r.sleep()
        
        
        rospy.loginfo('%s: Succeeded' % self._action_name)
        self._result.finished_tasks=self._todo
        self._as.set_succeeded(self._result)
        


In [2]:
rospy.init_node('ono')
server = OnoWorker("ono_tasks")



In [3]:
import rospy
#from __future__ import print_function

# Brings in the SimpleActionClient
import actionlib

# Brings in the messages used by the fibonacci action, including the
# goal message and the result message.
import ipython_robot_prototyping.msg

def action_feedback(fb):
    print(fb)

def ono_client(list_of_tasks):
    # Creates the SimpleActionClient, passing the type of the action
    # (FibonacciAction) to the constructor.
    client = actionlib.SimpleActionClient('ono_tasks', ipython_robot_prototyping.msg.workingAction)

    # Waits until the action server has started up and started
    # listening for goals.
    client.wait_for_server()

    # Creates a goal to send to the action server.
    goal = ipython_robot_prototyping.msg.workingGoal(list_of_orders=list_of_tasks)

    # Sends the goal to the action server.
    client.send_goal(goal, feedback_cb=action_feedback)

    # Waits for the server to finish performing the action.
    #client.wait_for_result() # we can wait for the result

    # Prints out the result of executing the action
    return client
    #return client.get_result()  # A FibonacciResult

#if __name__ == '__main__':
#    try:
#        # Initializes a rospy node so that the SimpleActionClient can
#        # publish and subscribe over ROS.
#        rospy.init_node('fibonacci_client_py')
result = ono_client(["eat", "sleep", "have fun"])
#print("Result:", ', '.join([str(n) for n in result.sequence]))
#except rospy.ROSInterruptException:
#print("program interrupted before completion", file=sys.stderr)

[INFO] [1581197104.501478]: Received list of goals
[INFO] [1581197104.501943]: Starting task execution
current_task: "eat"
current_task: "sleep"
current_task: "have fun"


In [6]:
result = ono_client(["eat", "sleep", "have fun"])


#fini=result.get_result()
#fini.finished_tasks

[INFO] [1581197120.483546]: Received list of goals
[INFO] [1581197120.484241]: Starting task execution
current_task: "eat"
[INFO] [1581197121.484625]: ono_tasks: Preempted
[INFO] [1581197121.585305]: Received list of goals
[INFO] [1581197121.585545]: Starting task execution
current_task: "eat"
current_task: "sleep"
current_task: "have fun"
current_task: "eat more"
[INFO] [1581197125.586386]: ono_tasks: Succeeded


In [7]:
result = ono_client(["eat", "sleep", "have fun", "eat more"])

result.cancel_goal()

[INFO] [1581197144.891156]: Received list of goals
[INFO] [1581197144.891708]: Starting task execution
[INFO] [1581197144.892287]: ono_tasks: Preempted


In [14]:
# Challenge 1

#Modify the action server so that for each task, robot responds a sentence "I have finished {} task"
#Modyfy the function so that result also shows date and time when the goal was accomplished
#You can use function similar to the one below
# You need to restart this notebook for changes to work. 
import time, datetime
timestamp = datetime.datetime.fromtimestamp(time.time())
print(timestamp)

2020-02-08 21:55:49.365397


## More details

Action file is required for actionlib

The format is

In [17]:
example_action="""
# goal part
int32 x 
int32 y
----
#result
string success
----
#feedback
int32 x
int32 y
"""

with open("../../action/turtlenavigation.action", "w") as turtlenavigation:
    turtlenavigation.write(example_action)

In [18]:
!cd ../../../../; catkin_make

Base path: /home/ubuntu/catkin_ws
Source space: /home/ubuntu/catkin_ws/src
Build space: /home/ubuntu/catkin_ws/build
Devel space: /home/ubuntu/catkin_ws/devel
Install space: /home/ubuntu/catkin_ws/install
####
#### Running command: "make cmake_check_build_system" in "/home/ubuntu/catkin_ws/build"
####
-- Using CATKIN_DEVEL_PREFIX: /home/ubuntu/catkin_ws/devel
-- Using CMAKE_PREFIX_PATH: /home/ubuntu/catkin_ws/devel;/opt/ros/kinetic
-- This workspace overlays: /home/ubuntu/catkin_ws/devel;/opt/ros/kinetic
-- Found PythonInterp: /usr/bin/python2 (found suitable version "2.7.12", minimum required is "2") 
-- Using PYTHON_EXECUTABLE: /usr/bin/python2
-- Using Debian Python package layout
-- Using empy: /usr/bin/empy
-- Using CATKIN_ENABLE_TESTING: ON
-- Call enable_testing()
-- Using CATKIN_TEST_RESULTS_DIR: /home/ubuntu/catkin_ws/build/test_results
-- Found gtest sources under '/usr/src/gmock': gtests will be built
-- Found gmock sources under '/usr/src/gmock': gmock will be built
-- Foun

[ 38%] Generating EusLisp code from ipython_robot_prototyping/turtlenavigationAction.msg
[ 48%] Built target ipython_robot_prototyping_generate_messages_cpp
[ 50%] Generating EusLisp code from ipython_robot_prototyping/turtlenavigationActionGoal.msg
[ 60%] Built target ipython_robot_prototyping_generate_messages_nodejs
[ 61%] Generating Python from MSG ipython_robot_prototyping/turtlenavigationActionGoal
[ 62%] Generating EusLisp code from ipython_robot_prototyping/turtlenavigationGoal.msg
[ 73%] Built target ipython_robot_prototyping_generate_messages_lisp
[ 74%] Generating Python from MSG ipython_robot_prototyping/turtlenavigationAction
[ 75%] Generating Python from MSG ipython_robot_prototyping/turtlenavigationGoal
[ 87%] Built target ipython_robot_prototyping_generate_messages_eus
[ 88%] Generating Python msg __init__.py for ipython_robot_prototyping
[ 89%] Generating Python srv __init__.py for ipython_robot_prototyping
[100%] Built target ipython_robot_prototyping_generate_message

In [ ]:
#Cmake Files need to be changed to include the action (the turtlenavigation)

add_action_files(
    FILES
   working.action
   turtlenavigation.action
#   Action2.action
 )

#After compiling multiple Python, (as well as Javascript, Java, Lisp, C++) modules are available such 
#as turtlenavigationAction turtlenavigationResult etc


In [ ]:
# Challenge 2

class GoToPointTurtle(object):
    _feedback = ipython_robot_prototyping.msg.turtlenavigationFeedback()
    _result = ipython_robot_prototyping.msg.turtlenavigationResult()
    
    def __init__(self,name):
        self._action_name = name
        self._action_server = actionlib.SimpleActionServer(self._action_name, 
                                                           ipython_robot_prototyping.msg.turtlenavigationAction,
                                                           execute_cb=self.execute_cb,
                                                           auto_start = False)
        self._as= self._action_server
        
        self._action_server.start()
        
    def execute_cb(self, goal):
      #PUT HERE a simple controller that moves robot towards goal 
      #sends feedback during the movement and "success" when the goal is reached
        
